In [ ]:
pip install blanc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 1.13.1 whic

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from blanc import BlancHelp
import os
import pandas as pd
import re
import torch
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

# Initialize BlancHelp for evaluation
blanc_help = BlancHelp()

# Define preprocessing function
def preprocess_text(text):
    # Preprocessing steps
    text = re.sub(r'(\d\d\d|\d\d|\d)\.\s', ' ', text)  # Removes paragraph labels like "1."
    text = re.sub(r'(?<=[a-zA-Z])\.(?=\d)', '', text)  # Removes dots in phrases like "File No.1063"
    text = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s[\da-z])', ' ', text)  # Removes ending dots of abbreviations
    text = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s?[\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~])', '', text)  # Removes ending dots of abbreviations
    text = re.sub(r'(?<!\.)[\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)  # Removes other punctuations
    text = text.replace("  "," ")  # Remove double spaces
    tokens = nltk.word_tokenize(text)  # Tokenize
    tokens = [token.lower() for token in tokens]  # Lowercase
    stop_words = stopwords.words('english')  # Define stopwords
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    return ' '.join(tokens)


# Define chunking function
def chunk_text(text, chunk_size):
    sentences = sent_tokenize(text)
    chunks = [sentences[i:i+chunk_size] for i in range(0, len(sentences), chunk_size)]
    return chunks

# Define summarization function
def summarize_chunk(chunk, model_name="facebook/bart-large-cnn", device=device):
    summarizer = pipeline("summarization", model=model_name, device=device)
    max_length = min(len(chunk) // 2, 100)  # Set max_length dynamically
    return summarizer(chunk, max_length=max_length, min_length=10, do_sample=False)[0]['summary_text']


# Define reverse summarization function
def reverse_summarization(text, chunk_size=5, model_name="facebook/bart-large-cnn"):
    preprocessed_text = preprocess_text(text)
    chunks = chunk_text(preprocessed_text, chunk_size)
    chunk_summaries = [summarize_chunk(' '.join(chunk), model_name) for chunk in chunks]
    final_summary = ' '.join(chunk_summaries)
    return final_summary

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Define directory paths
base_dir = "/content/drive/MyDrive/to be delete"
judgement_dir = os.path.join(base_dir, "judgment")
summary_dir = os.path.join(base_dir, "summary")

# Initialize a list to store results
results = []

# Iterate through each file in the judgement directory
for file in os.listdir(judgement_dir):
    if file.endswith(".txt"):
        # Read judgement text
        with open(os.path.join(judgement_dir, file), 'r', encoding='utf-8') as f:
            judgement_text = f.read()

        # Generate summary
        generated_summary = reverse_summarization(judgement_text)
        print("Done summary {}\n".format(file))

        # Evaluate summary with BlancHelp
        blanc_score = blanc_help.eval_once(judgement_text, generated_summary)
        print("Done Blanc evaluation {}\n".format(file))

        # Append results
        results.append({
            'File': file,
            'Blanc_Score': blanc_score
        })
        print("Done append {}\n".format(file))

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Save results to Excel
results_df.to_excel("/content/drive/MyDrive/dataset/IN-Abs/test-data/reverse_summarization.xlsx", index=False)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Done Blanc evaluation 6728.txt

Done append 6728.txt



Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is set to 100, but your input_length is only 80. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is set to 100, but your input_length is only 74. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your

Done summary 78.txt



100%|██████████| 336/336 [14:12<00:00,  2.54s/it]


Done Blanc evaluation 78.txt

Done append 78.txt



# Saving only summaries of this recursive summarization

In [ ]:
import os
import pandas as pd
from nltk.tokenize import sent_tokenize
import re
import torch
from nltk.corpus import stopwords
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1  # 0 for GPU, -1 for CPU

In [ ]:

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'(\d\d\d|\d\d|\d)\.\s', ' ', text)  # Removes paragraph labels
    text = re.sub(r'(?<=[a-zA-Z])\.(?=\d)', '', text)  # Removes dots in phrases
    text = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s[\da-z])', ' ', text)  # Removes ending dots of abbreviations
    text = re.sub(r'(?<=\d|[a-zA-Z])\.(?=\s?[\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~])', '', text)  # Removes ending dots of abbreviations
    text = re.sub(r'(?<!\.)[\!\"\#\$\%\&\'\(\)\*\+\,\-\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~]', ' ', text)  # Removes other punctuations
    text = text.replace(" ", " ")  # Remove double spaces
    tokens = nltk.word_tokenize(text.lower())
    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Chunking function
def chunk_text(text, chunk_size):
    sentences = sent_tokenize(text)
    chunks = [sentences[i:i+chunk_size] for i in range(0, len(sentences), chunk_size)]
    return chunks

# Summarization function
def summarize_chunk(chunk, model_name="facebook/bart-large-cnn", device=device):
    summarizer = pipeline("summarization", model=model_name, device=device)
    max_length = min(len(chunk) // 2, 100)  # Set max_length dynamically
    return summarizer(chunk, max_length=max_length, min_length=10, do_sample=False)[0]['summary_text']

# Recursive summarization function
def recursive_summarization(text, chunk_size=5, model_name="facebook/bart-large-cnn"):
    preprocessed_text = preprocess_text(text)
    chunks = chunk_text(preprocessed_text, chunk_size)
    chunk_summaries = [summarize_chunk(' '.join(chunk), model_name) for chunk in chunks]
    final_summary = ' '.join(chunk_summaries)

    return final_summary

# Define directory paths
base_dir = "/content/drive/MyDrive/to be delete"
judgement_dir = os.path.join(base_dir, "judgment")

summaries_dir = os.path.join(base_dir, "summaries")
# Create the summaries directory if it doesn't exist
if not os.path.exists(summaries_dir):
    os.makedirs(summaries_dir)

# Iterate through each file in the judgement directory
for file in os.listdir(judgement_dir):
    if file.endswith(".txt"):
        # Read judgement text
        with open(os.path.join(judgement_dir, file), 'r', encoding='utf-8') as f:
            judgement_text = f.read()

        # Generate summary using recursive summarization
        summary = recursive_summarization(judgement_text)

        # Save summary to separate Excel file
        output_file = os.path.join(base_dir, "summaries", file.replace(".txt", "_summary.xlsx"))
        summary_df = pd.DataFrame({'Summary': [summary]})
        summary_df.to_excel(output_file, index=False)

        print(f"Summary saved to '{output_file}'")


Your max_length is set to 100, but your input_length is only 72. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 100, but your input_length is only 59. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 12, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Summary saved to '/content/drive/MyDrive/to be delete/summaries/4807_summary.xlsx'


Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 53. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)
Your max_length is set to 100, but your input_length is only 70. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)
Your

Summary saved to '/content/drive/MyDrive/to be delete/summaries/2796_summary.xlsx'


Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


...

Summary saved to '/content/drive/MyDrive/to be delete/summaries/6245_summary.xlsx'


Your max_length is set to 81, but your input_length is only 54. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)
Your max_length is set to 100, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 98. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)
Your 

Summary saved to '/content/drive/MyDrive/to be delete/summaries/6728_summary.xlsx'


Your max_length is set to 100, but your input_length is only 81. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)
Your max_length is set to 100, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 51. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your

Summary saved to '/content/drive/MyDrive/to be delete/summaries/78_summary.xlsx'
